# 手書き数字認識
### Kerasに入っているデータセットMNISTを使って手書きの0～9の数字を認識する
### Keras
ディープラーニングのライブラリ　バックエンドでTensorFlowを使用（デフォルト、Theanoなどに変更もできる）
- 公式リファレンス https://keras.io/ja/
- バックエンドの変更 https://qiita.com/samacoba/items/3e29f0ac61c2f369892b

#### ライブラリのバージョン
- python 3.7.0 ※3.8.1も動作確認済み
- keras 2.3.1
- numpy 1.17.3
- opencv 4.1.1　※pipの場合はopencvのみでOKです
- matplotlib 3.1.2

In [ ]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from matplotlib import pyplot as plt
%matplotlib inline

### データセット読み込み

In [ ]:
# (学習画像,学習ラベル),(テスト画像,テストラベル)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# 28×28の画像
x_train.shape

In [ ]:
# MNIST最初の手書き数字画像は5
five = x_train[0]
plt.imshow(five.reshape(28,28))
plt.show()

### 画像データの変換

In [ ]:
# 行数を保存
num_train_data = x_train.shape[0]
num_test_data = x_test.shape[0]

In [ ]:
# 画像の行と列を掛け算して一次元配列にする時の要素数を求める
# 28×28なので784
num_flatten_data = x_train.shape[1] * x_train.shape[2]

# 画像データを行列に変換
x_train = x_train.reshape(num_train_data, num_flatten_data)
x_test = x_test.reshape(num_test_data, num_flatten_data)

In [ ]:
# グレースケールにする
num_gray_scale_max = 255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= num_gray_scale_max
x_test /= num_gray_scale_max

In [ ]:
# one-hotエンコード
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### モデル構築
MNISTは単純な画像のため層数が少なくても高い精度が出ます

In [ ]:
model = Sequential()

# ノード512個、活性化関数ReLU、入力値は一次配列で784要素（28×28）
model.add(Dense(512, activation='relu', input_shape=(784,)))

# 20％の入力ノードを無効化
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

# 0～9の10クラス分類
model.add(Dense(10, activation='softmax'))

# モデルの精度を評価する方法を設定
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# モデルの構成を表示
model.summary()

### 学習・テスト実行
先ほど構築したモデルに学習データ、テストデータを投入して0～9の数字判定ができているかをチェックする<br>
**※処理に時間が掛かるので注意**
- バッチサイズ：学習画像を何枚ずつに分けて学習させるか（外れ値の影響を少なくする）
- エポック数：学習する回数（学習画像の総枚数/バッチサイズしたn個のサブセットが全部学習して1エポック）

In [ ]:
batch_size = 128
epochs = 20

# 学習画像、学習ラベル、バッチサイズ、エポック数、進捗を表示、テストデータ
history = model.fit(x=x_train, 
                    y=y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_data=(x_test,y_test))

### 学習・テスト結果をグラフにして表示
- 青線：学習データの正解率
- 赤線：テストデータの正解率

In [ ]:
# グラフのサイズ
plt.figure(figsize=(10,6))

# 青線：学習結果
plt.plot(history.history['accuracy'], color='b')
# 赤線：テスト結果
plt.plot(history.history['val_accuracy'], color='r')

plt.tick_params(labelsize=18)
plt.title('epoch - accuracy graph')
# 縦軸：正解率（モデルの精度）
plt.ylabel('accuracy')
# 横軸：エポック数
plt.xlabel('epoch')
plt.legend(['train','test'])
plt.show()